# RSI Breakout Strategy Backtest

In [ ]:

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:

# Download stock data
data = yf.download("AAPL", start="2019-01-01", end="2023-01-01")


In [ ]:

# Calculate RSI
delta = data['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
data['RSI'] = 100 - (100 / (1 + rs))
data = data.dropna()


In [ ]:

# Create RSI breakout signals
data['Signal'] = 0
data.loc[(data['RSI'].shift(1) < 30) & (data['RSI'] > 30), 'Signal'] = 1  # Buy
data.loc[(data['RSI'].shift(1) > 70) & (data['RSI'] < 70), 'Signal'] = -1  # Sell
data['Position'] = data['Signal'].shift(1)


In [ ]:

# Calculate returns
data['Daily Return'] = data['Close'].pct_change()
data['Strategy Return'] = data['Daily Return'] * data['Position']


In [ ]:

# Plot RSI and price
plt.figure(figsize=(14, 6))
plt.plot(data['RSI'], label='RSI')
plt.axhline(70, color='r', linestyle='--', label='Overbought (70)')
plt.axhline(30, color='g', linestyle='--', label='Oversold (30)')
plt.title('AAPL RSI Indicator')
plt.xlabel('Date')
plt.ylabel('RSI')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# Plot cumulative returns
data[['Daily Return', 'Strategy Return']].dropna().cumsum().plot(figsize=(12, 6), title='Cumulative Returns: RSI Breakout Strategy')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.grid(True)
plt.show()
